In [ ]:
# Plot GPS data exported from the phone app

import pandas as pd


GPS_DATA_PATH = "gps_phone_data.csv"

# Read the CSV file
df = pd.read_csv(GPS_DATA_PATH, parse_dates=["loggingTime(txt)"])

# Discard rows without GPS data
df_gps = df[
    [
        "loggingSample(N)",
        "locationTimestamp_since1970(s)",
        "locationLatitude(WGS84)",
        "locationLongitude(WGS84)",
    ]
]
df_gps = df_gps.dropna(subset=["locationLatitude(WGS84)", "locationLongitude(WGS84)"])
df_gps = df_gps.reset_index(drop=True)

first_sample_time = df_gps["locationTimestamp_since1970(s)"].iloc[0]
df_gps["timeSinceStart(s)"] = df_gps["locationTimestamp_since1970(s)"] - first_sample_time

# Rename df columns
df_gps = df_gps.rename(
    columns={
        "loggingSample(N)": "loggingSample",
        "locationTimestamp_since1970(s)": "timestamp",
        "locationLatitude(WGS84)": "latitude",
        "locationLongitude(WGS84)": "longitude",
    }
)

# Display the first few rows of the DataFrame
print(df_gps.head())

In [ ]:
# Plot GPS data logged by the pose estimator subscriber

import pandas as pd
import json


GPS_DATA_PATH = "gps_heading_test_2.data"

# Open and read the file
data = []
with open(GPS_DATA_PATH, 'r') as file:
    for line in file:
        json_data = json.loads(line.strip())
        data.append(json_data)

# Create a DataFrame from the list of dictionaries
df_gps = pd.DataFrame(data)

# Display the first few rows of the DataFrame
print(df_gps.head())

In [ ]:
# Visualize the GPS data on a map

import folium


# Create a map centered on the mean latitude and longitude
center_lat = df_gps["latitude"].mean()
center_lon = df_gps["longitude"].mean()
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=21,
    max_zoom=22,
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
)

folium.TileLayer("OpenStreetMap").add_to(m)
folium.LayerControl().add_to(m)

# Add markers for each GPS point
for idx, row in df_gps.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        # popup=f"Sample: {row['loggingSample']}",
        tooltip=f"Time delta: {row['timestamp']}",
    ).add_to(m)

# Add a PolyLine to connect the points
locations = df_gps[["latitude", "longitude"]].values.tolist()
folium.PolyLine(locations, weight=2, color="red").add_to(m)

# Save the map
# m.save("gps_data_map.html")

# Display the map
m

In [ ]:
import folium
import json
import pandas as pd
from math import radians, sin, cos

import json


GPS_DATA_PATH = "gps_compass_data_2.json"

# Read json file
with open(GPS_DATA_PATH, 'r') as file:
    data_raw = json.load(file)

data = []
for d in data_raw:
    if "locationLatitude" in d:
        data.append(d)

df_gps = pd.DataFrame(data)

df_gps["locationLatitude"] = df_gps["locationLatitude"].astype(float)
df_gps["locationLongitude"] = df_gps["locationLongitude"].astype(float)
df_gps["locationTrueHeading"] = df_gps["locationTrueHeading"].astype(float)

# Create a map centered on the mean latitude and longitude
center_lat = df_gps["locationLatitude"].mean()
center_lon = df_gps["locationLongitude"].mean()
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=21,
    max_zoom=22,
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
)

folium.TileLayer("OpenStreetMap").add_to(m)
folium.LayerControl().add_to(m)

# Function to create an arrow
def create_arrow(location, heading, color='red', size=5):
    angle = radians(float(heading))
    end_lat = location[0] + size * 0.00001 * cos(angle)
    end_lon = location[1] + size * 0.00001 * sin(angle)

    folium.PolyLine(
        locations=[location, (end_lat, end_lon)],
        color=color,
        weight=2,
        arrow=True
    ).add_to(m)

# Add markers and arrows for each data point
for index, point in df_gps.iterrows():
    folium.Marker(
        [point['locationLatitude'], point['locationLongitude']],
        # popup=f"Sample: {point['loggingSample']}<br>Time: {point['loggingTime']}"
    ).add_to(m)

    create_arrow(
        (point['locationLatitude'], point['locationLongitude']),
        point['locationTrueHeading']
    )

# Display the map
m